In [1]:
import pandas as pd
import numpy as np
import pdb
import time
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, GridSearchCV
#from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, balanced_accuracy_score
from sklearn.neighbors import KNeighborsRegressor
import tensorflow as tf
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
#from xgboost import XGBClassifier
import lightgbm as gbm
from sklearn.decomposition import PCA
import os
import pickle
%matplotlib nbagg
from sklearn.cluster import KMeans

C:\Users\Renganathan\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
X_test = pd.read_csv('X_test_PCA.csv', index_col=0)
cluster_model = pickle.load(open('New Models/clustering.sav', 'rb'))
cluster_y = cluster_model.predict(X_test)

X_test1 = X_test.iloc[np.where(cluster_y == 0)[0]]
X_test2 = X_test.iloc[np.where(cluster_y == 1)[0]]

In [3]:
def predict(test_data, models, thresh, weights=None):
    
    if weights == None:
        weights = np.ones(len(models))
    
    N = len(models)
    ypred_final = np.zeros(test_data.shape[0])
    
    for i in range(N):
        if thresh[i]==None:
            ypred_final += weights[i]*models[i].predict(test_data)
        
        else:
            y_test_prob = models[i].predict(test_data)    
            y_pred_test = np.zeros(len(y_test_prob))
            y_pred_test[np.argwhere(y_test_prob>thresh[i])] = 1 
            
            ypred_final += weights[i]*y_pred_test
            
       
    ypred_final = ypred_final/sum(weights)
    
    y_hat_test = np.zeros(test_data.shape[0])
    y_hat_test[np.argwhere(ypred_final>0.5)] = 1
    
    y = pd.DataFrame(y_hat_test, index=test_data.index)
    
    return(y)

In [14]:
models1 = pickle.load(open('Dummy/models11.sav', 'rb'))
models2 = pickle.load(open('Dummy/models22.sav', 'rb'))

thresh1 = pickle.load(open('Dummy/threshold11.sav', 'rb'))
thresh2 = pickle.load(open('Dummy/threshold22.sav', 'rb'))

In [15]:
y_hat_test1 = predict(X_test1, models1, thresh1)
y_hat_test2 = predict(X_test2, models2, thresh2)

In [16]:
y = [y_hat_test1, y_hat_test2]
y_hat_test = pd.concat(y)
y_hat_test = y_hat_test.reindex(X_test.index)

In [17]:
y_hat_test.head()

,0
application_key,
578069,1.0
578070,0.0
578071,0.0
578072,0.0
578073,0.0


In [18]:
X_test.index

Int64Index([578069, 578070, 578071, 578072, 578073, 578074, 578075, 578076,
            578077, 578078,
            ...
            310022, 310023, 310024, 310025, 310026, 310027, 310028, 310029,
            310030, 310031],
           dtype='int64', name='application_key', length=47000)

In [19]:
y_hat_test.to_csv('Group7_6.csv', header=False)